# Titanic dataset classification

## Setup

In [ ]:
using Revise
using Titanic
using DataFrames
using Statistics
using StatsBase
using StatsPlots
using Plots
using Interact
using Blink
gr()
theme(:lime)

In [ ]:
df = read_csv_data("../data/train.csv")
describe(df)

## Data (pre-)processing

* target variable
    * survived
* ordinal properties(numerical)
    * pclass
    * age
    * sibsp
    * parch
    * fare
* nominal properties:
    * sex
    * name
    * ticket
    * cabin
    * home.dest
    * embarked

### Couple of graphs

In [ ]:
@df df histogram(:Age, bins = :scott, fillalpha = 0.4)

In [ ]:
gr(size = (600, 500))
@df df corrplot([:Survived :Fare], grid = false)

### Missing values

how many values do we miss in each column?

In [ ]:
Dict(map(col -> (col, (sum(completecases(df,col))/nrow(df), nrow(df) - sum(completecases(df,col)))), names(df)))

replace missing in Age with median

In [ ]:
col_median = apply_to_cols(df, :Age, median)
df = replace_in_cols(df, :Age, missing, col_median)

replace missing in Embarked with the most common city

In [ ]:
col_city = apply_to_cols(df, :Embarked, most_common)
df = replace_in_cols(df, :Embarked, missing, col_city)

replace missing in Cabin with unknown and strip numbers of cabins leaving only its class

In [ ]:
unique(map(entry -> entry[begin],skipmissing(df[!, :Cabin])))

use 'N' for replacement

In [ ]:
df = replace_in_cols(df, :Cabin, missing, "N") 
col_cabin = apply_to_cols(df, :Cabin, strip_cabin_numbers)
df[!, :Cabin] = col_cabin
df

### Names to titles

In [ ]:
groups = get_title_groups()
df = replace_names_with_title_categories(df, groups)

### Remove columns

In [ ]:
df = select(df, Not([:PassengerId, :Ticket]))

### Nominal to Categorical

In [ ]:
df = categorize(df)

### Corr plot

In [ ]:
gr(size = (1000, 1000))
@df df corrplot([:Survived :Fare :Age :Sex :Cabin], grid = false)

### Correlation matrix

In [ ]:
heatmap(cor(Matrix(df)); ticks=(1:length(names(df)), names(df)), xrotation=30, yflip=true)

### Standartize

In [ ]:
survived = df[!, :Survived]
df = standartize(df)
df.Survived = survived
df

## Splitting the dataset

In [ ]:
trn, val, tst = random_split(df, [0.6, 0.2, 0.2])

## Models

### K-nn

In [ ]:
knn = K_nn(;n=50, metric="l2")
model_fit(knn, trn[!, [:Fare, :Sex]], trn[!, :Survived])
val_X, val_y = val[!, [:Fare, :Sex]], val[!, :Survived]
val_preds = model_predict(knn, val_X)
accuracy_my(val_y, val_preds)

In [ ]:
tst_X, tst_y = tst[!, [:Fare, :Sex]], tst[!, :Survived]
accuracy_my(model_predict(knn, tst_X), tst_y)

### Logistic regression

In [ ]:
logreg = Log_reg()
trn_X, trn_y = trn[!, Not(:Survived)], trn[!, :Survived]
w, wt, Lt = model_fit(logreg, trn_X, trn_y, ones(size(trn_X)[end]))
val_X, val_y = val[!, Not(:Survived)], val[!, :Survived]
accuracy_my(model_predict(logreg, val_X), val_y)

In [ ]:
tst_X, tst_y = tst[!, Not(:Survived)], tst[!, :Survived]
accuracy_my(model_predict(logreg, tst_X), tst_y)


### NN

In [ ]:
args = Args()
train_data, val_data, test_data = to_batches(trn, val, tst, args)
m = train(train_data, val_data)

In [ ]:
println("Test accuracy: ", accuracy(test_data, m))

### 

## Final evaluation and submission